In [1]:
!pip install transformers
!pip install evaluate

In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.optim import AdamW
from datasets import load_dataset


checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


# dataset load
raw_dataset = load_dataset("glue","mrpc")

raw_train_data = raw_dataset["train"]

print(raw_train_data.features)


def tokenizer_functoin(example):
    return tokenizer(example["sentence1"],example["sentence2"],truncation=True,padding='max_length',max_length=128)


tokenized_dataset = raw_dataset.map(tokenizer_functoin,batched=True)

print(tokenized_dataset)

# we have to type of padding
# fixed size padding
# dynamic size padding during the batch creating to max size sentence in the batch

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

{'sentence1': Value('string'), 'sentence2': Value('string'), 'label': ClassLabel(names=['not_equivalent', 'equivalent']), 'idx': Value('int32')}


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


In [3]:
tokenized_dataset = tokenized_dataset.remove_columns(["sentence1","sentence2","idx"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset = tokenized_dataset.with_format("torch")


In [4]:
tokenized_dataset["train"][0]["input_ids"].shape

torch.Size([128])

In [5]:
tokenized_dataset["train"][130]["input_ids"].shape

torch.Size([128])

In [6]:
# create batches of the data
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_dataset["train"],shuffle=True,batch_size=16)

In [7]:
train_dataloader

In [8]:
#
for index,batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if index > 5:
      break

torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])


In [9]:
# dynamic padding now

def tokenizer_functoin(example):
    return tokenizer(example["sentence1"],example["sentence2"],truncation=True)


tokenized_dataset_dynamic_padding = raw_dataset.map(tokenizer_functoin,batched=True)
tokenized_dataset_dynamic_padding = tokenized_dataset_dynamic_padding.remove_columns(["sentence1","sentence2","idx"])
tokenized_dataset_dynamic_padding = tokenized_dataset_dynamic_padding.rename_column("label","labels")
tokenized_dataset_dynamic_padding = tokenized_dataset_dynamic_padding.with_format("torch")


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader_dynamic = DataLoader(tokenized_dataset_dynamic_padding["train"], shuffle=True, batch_size=16, collate_fn=data_collator)


for index,batch in enumerate(train_dataloader_dynamic):
    print(batch["input_ids"].shape)
    if index > 5:
      break

torch.Size([16, 81])
torch.Size([16, 87])
torch.Size([16, 68])
torch.Size([16, 89])
torch.Size([16, 71])
torch.Size([16, 83])
torch.Size([16, 82])


In [11]:
len(train_dataloader_dynamic)
# 3665/16

230

In [12]:
len(train_dataloader)

230

In [13]:
tokenized_dataset["train"][0]

{'labels': tensor(1),
 'input_ids': tensor([  101,  2572,  3217,  5831,  5496,  2010,  2567,  1010,  3183,  2002,
          2170,  1000,  1996,  7409,  1000,  1010,  1997,  9969,  4487, 23809,
          3436,  2010,  3350,  1012,   102,  7727,  2000,  2032,  2004,  2069,
          1000,  1996,  7409,  1000,  1010,  2572,  3217,  5831,  5496,  2010,
          2567,  1997,  9969,  4487, 23809,  3436,  2010,  3350,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer",
                                eval_strategy="epoch",
                                learning_rate=2e-5,
                                num_train_epochs=2,
                                weight_decay=0.01,
                                per_device_train_batch_size=16,
                                per_device_eval_batch_size=16,
                                  )

In [17]:
# import wandb
# ‍
# # 1. Define which wandb project to log to and name your run
# run = wandb.init(project="gpt-5",
# run_name="gpt-5-base-high-lr")
# ‍
# # 2. Add wandb in your `TrainingArguments`
# args = TrainingArguments(..., report_to="wandb")
# ‍
# # 3. W&B logging will begin automatically when your start training your Trainer
# trainer = Trainer(..., args=args)
# trainer.train()

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_dynamic_padding["train"],
    eval_dataset=tokenized_dataset_dynamic_padding["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,

)

In [18]:
trainer.train()
# 73c9cba4aa03060ce4cd30ba7733ba3390e61b67

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azheraly009 (azheraly009-nust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.435022
2,No log,0.405791


TrainOutput(global_step=460, training_loss=0.46470947265625, metrics={'train_runtime': 157.9322, 'train_samples_per_second': 46.45, 'train_steps_per_second': 2.913, 'total_flos': 285847550542320.0, 'train_loss': 0.46470947265625, 'epoch': 2.0})

In [23]:
predictions = trainer.predict(tokenized_dataset_dynamic_padding["validation"])
# the model return is logit which are raw we have to compute it into the probality by using sigmoid,softmax
predictions.predictions

array([[-1.45424092e+00,  2.43165779e+00],
       [ 4.73370135e-01, -9.70458448e-01],
       [ 5.38205147e-01, -8.89596581e-01],
       [-7.06623077e-01,  6.11091554e-01],
       [ 3.99594903e-01, -9.15792346e-01],
       [-9.38397706e-01,  1.67666137e+00],
       [-9.89454269e-01,  1.54492521e+00],
       [-1.23730671e+00,  2.05166316e+00],
       [-1.03170729e+00,  1.93519104e+00],
       [-1.32474720e+00,  2.30547500e+00],
       [-1.43166161e+00,  2.31322670e+00],
       [ 1.56190008e-01, -5.08755445e-01],
       [ 6.45844042e-01, -1.14323223e+00],
       [-9.28546369e-01,  1.40919220e+00],
       [-1.31706321e+00,  2.36360741e+00],
       [-1.10116625e+00,  1.75417149e+00],
       [-1.48463798e+00,  2.44432855e+00],
       [ 6.14429414e-01, -1.01238823e+00],
       [-1.46717286e+00,  2.46458054e+00],
       [ 6.50253594e-01, -1.06420386e+00],
       [ 5.15344441e-01, -1.06127691e+00],
       [-7.04758823e-01,  1.22540355e+00],
       [-1.10919312e-01, -2.75352389e-01],
       [-1.

In [31]:
predictions.label_ids[:20]

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0])

In [32]:
tokenized_dataset_dynamic_padding["validation"][:20]["labels"]

tensor([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0])

In [34]:
import numpy as np
import evaluate
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8308823529411765, 'f1': 0.8824531516183987}

In [33]:
# model eveluation


In [35]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [50]:
logits = predictions.predictions
y_pred =  torch.sigmoid(torch.tensor(logits))
y_pred = torch.argmax(y_pred,dim=1)
y_true = torch.tensor(predictions.label_ids)

In [51]:
y_pred[:10]

tensor([1, 0, 0, 1, 0, 1, 1, 1, 1, 1])

In [52]:
y_true[:10]

tensor([1, 0, 0, 1, 0, 1, 0, 1, 1, 1])

In [54]:
accuracy_fn(y_true,y_pred)

83.08823529411765

In [55]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [56]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset_dynamic_padding["train"],
    eval_dataset=tokenized_dataset_dynamic_padding["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [57]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.457619,0.852941,0.893617
2,No log,0.562432,0.845588,0.891938


TrainOutput(global_step=460, training_loss=0.20398451763650646, metrics={'train_runtime': 131.7023, 'train_samples_per_second': 55.701, 'train_steps_per_second': 3.493, 'total_flos': 285847550542320.0, 'train_loss': 0.20398451763650646, 'epoch': 2.0})